In [1]:
%load_ext sql

In [2]:
%sql sqlite:///C:\Users\User\Desktop\sql\chinook.db

In [3]:
%%sql
SELECT *
FROM sqlite_master


 * sqlite:///C:\Users\User\Desktop\sql\chinook.db
Done.


type,name,tbl_name,rootpage,sql
table,albums,albums,2,"CREATE TABLE ""albums"" ( [AlbumId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [Title] NVARCHAR(160) NOT NULL, [ArtistId] INTEGER NOT NULL, FOREIGN KEY ([ArtistId]) REFERENCES ""artists"" ([ArtistId]) ON DELETE NO ACTION ON UPDATE NO ACTION )"
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,artists,artists,4,"CREATE TABLE ""artists"" ( [ArtistId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [Name] NVARCHAR(120) )"
table,customers,customers,5,"CREATE TABLE ""customers"" ( [CustomerId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [FirstName] NVARCHAR(40) NOT NULL, [LastName] NVARCHAR(20) NOT NULL, [Company] NVARCHAR(80), [Address] NVARCHAR(70), [City] NVARCHAR(40), [State] NVARCHAR(40), [Country] NVARCHAR(40), [PostalCode] NVARCHAR(10), [Phone] NVARCHAR(24), [Fax] NVARCHAR(24), [Email] NVARCHAR(60) NOT NULL, [SupportRepId] INTEGER, FOREIGN KEY ([SupportRepId]) REFERENCES ""employees"" ([EmployeeId]) ON DELETE NO ACTION ON UPDATE NO ACTION )"
table,employees,employees,8,"CREATE TABLE ""employees"" ( [EmployeeId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [LastName] NVARCHAR(20) NOT NULL, [FirstName] NVARCHAR(20) NOT NULL, [Title] NVARCHAR(30), [ReportsTo] INTEGER, [BirthDate] DATETIME, [HireDate] DATETIME, [Address] NVARCHAR(70), [City] NVARCHAR(40), [State] NVARCHAR(40), [Country] NVARCHAR(40), [PostalCode] NVARCHAR(10), [Phone] NVARCHAR(24), [Fax] NVARCHAR(24), [Email] NVARCHAR(60), FOREIGN KEY ([ReportsTo]) REFERENCES ""employees"" ([EmployeeId]) ON DELETE NO ACTION ON UPDATE NO ACTION )"
table,genres,genres,10,"CREATE TABLE ""genres"" ( [GenreId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [Name] NVARCHAR(120) )"
table,invoices,invoices,11,"CREATE TABLE ""invoices"" ( [InvoiceId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [CustomerId] INTEGER NOT NULL, [InvoiceDate] DATETIME NOT NULL, [BillingAddress] NVARCHAR(70), [BillingCity] NVARCHAR(40), [BillingState] NVARCHAR(40), [BillingCountry] NVARCHAR(40), [BillingPostalCode] NVARCHAR(10), [Total] NUMERIC(10,2) NOT NULL, FOREIGN KEY ([CustomerId]) REFERENCES ""customers"" ([CustomerId]) ON DELETE NO ACTION ON UPDATE NO ACTION )"
table,invoice_items,invoice_items,13,"CREATE TABLE ""invoice_items"" ( [InvoiceLineId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [InvoiceId] INTEGER NOT NULL, [TrackId] INTEGER NOT NULL, [UnitPrice] NUMERIC(10,2) NOT NULL, [Quantity] INTEGER NOT NULL, FOREIGN KEY ([InvoiceId]) REFERENCES ""invoices"" ([InvoiceId]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([TrackId]) REFERENCES ""tracks"" ([TrackId]) ON DELETE NO ACTION ON UPDATE NO ACTION )"
table,media_types,media_types,15,"CREATE TABLE ""media_types"" ( [MediaTypeId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [Name] NVARCHAR(120) )"
table,playlists,playlists,16,"CREATE TABLE ""playlists"" ( [PlaylistId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [Name] NVARCHAR(120) )"


In [5]:
%%sql
-- load tables in sql_master
SELECT 
    name
FROM 
    sqlite_master
WHERE
    type = "table"

 * sqlite:///C:\Users\User\Desktop\sql\chinook.db
Done.


name
albums
sqlite_sequence
artists
customers
employees
genres
invoices
invoice_items
media_types
playlists


In [48]:
%%sql
-- 1. Find the top-selling artist in each country
with ArtistSale as (
    SELECT 
        a.name as Name,
        round(sum(item.UnitPrice * item.Quantity), 3) as Total_sales,
        c.Country
    FROM 
        artists as a
    JOIN
        albums as al
    ON 
        a.ArtistId = al.ArtistId
    JOIN
        tracks as t
    ON
        al.AlbumId = t.AlbumId   
    JOIN
        invoice_items as item
    ON
        t.TrackId = item.TrackId  
    JOIN
        invoices as i
    ON 
        item.InvoiceId = i.InvoiceId
    JOIN
        customers as c
    ON
        i.CustomerId = c.CustomerId
    GROUP BY 
            a.name, 
            c.Country
)
SELECT 
    Name,
    country,
    Total_sales,
    Ranking
FROM
    (
        SELECT Name,
        country,
        Total_sales,
        DENSE_RANK() OVER(PARTITION BY Country ORDER BY Total_sales DESC) as Ranking
        FROM ArtistSale
        
    ) ranked

WHERE Ranking = 1
ORDER BY Total_sales DESC
limit 5

 * sqlite:///C:\Users\User\Desktop\sql\chinook.db
Done.


Name,country,Total_sales,Ranking
The Office,USA,33.83,1
Iron Maiden,Australia,17.82,1
Lost,Czech Republic,15.92,1
Lost,Ireland,15.92,1
Os Paralamas Do Sucesso,Canada,15.84,1


In [69]:
%%sql
-- 2. Calculate the running total of sales per year
SELECT 
    strftime("%Y", InvoiceDate) as year,
    sum(Total) as Running_cost
    
FROM invoices
GROUP BY strftime("%Y", InvoiceDate)
ORDER BY year


 * sqlite:///C:\Users\User\Desktop\sql\chinook.db
Done.


year,Running_cost
2009,449.46
2010,481.45
2011,469.58
2012,477.53
2013,450.58


In [88]:
%%sql
-- 4. Classify tracks as Short, Medium, or Long based on their duration
SELECT 
    Name as Track_Name,
    (Milliseconds/ 60000) Duration_Min,

    CASE 
        WHEN Milliseconds < 180000 THEN "Short"
        WHEN Milliseconds BETWEEN 180000 AND 300000 THEN "Medium" 
        ELSE "Long"
    END as Duration_category
FROM tracks
ORDER BY Duration_Min DESC
LIMIT 5

 * sqlite:///C:\Users\User\Desktop\sql\chinook.db
Done.


Track_Name,Duration_Min,Duration_category
Occupation / Precipice,88,Long
Through a Looking Glass,84,Long
"Battlestar Galactica, Pt. 1",49,Long
"Battlestar Galactica, Pt. 2",49,Long
The Man With Nine Lives,49,Long


In [115]:
%%sql
-- 5. Get the most popular genre for each country
    with GenreSales as (
    SELECT 
        g.Name as Genre,
        c.Country as Country,
        (item.UnitPrice * item.Quantity) as Revenue
        
    FROM 
        genres as g
    JOIN 
        tracks as t
    ON 
        g.GenreId = t.GenreId
    JOIN 
        invoice_items as item
    ON 
        t.TrackId = item.TrackId
    JOIN
        invoices as i
    ON
        item.InvoiceId = i.InvoiceId
    JOIN
        customers as c
    ON 
        i.CustomerId = c.CustomerId  
    GROUP BY 
        c.Country, g.Name
)
SELECT Genre, Country, Revenue, Ranking
FROM (
    SELECT Genre, Country, Revenue,
    ROW_NUMBER() OVER(PARTITION BY Country ORDER BY Revenue) as Ranking
    FROM GenreSales
)
WHERE Ranking = 1
ORDER BY Country
LIMIT 5

 * sqlite:///C:\Users\User\Desktop\sql\chinook.db
Done.


Genre,Country,Revenue,Ranking
Alternative & Punk,Argentina,0.99,1
Blues,Australia,0.99,1
Classical,Austria,0.99,1
Alternative & Punk,Belgium,0.99,1
Alternative & Punk,Brazil,0.99,1
